## Meanshift and Camshift (均值漂移和Camshift)
## 1. Meanshift
### 1.1 均值漂移的原理
均值漂移是一种基于概率密度的迭代算法，它的基本思想是：对于给定的一定数量样本，任选其中一个样本，以该样本为中心点划定一个圆形区域，求取该区域内样本的质心，即密度中心（mean），然后以质心为新的中心点重复上述操作，直至最终收敛。均值漂移算法的收敛性较好，且与初始值无关，但是其计算复杂度较高，对初始值较为敏感，且只能得到一个局部极值。
### 1.2 均值漂移的应用
均值漂移算法在图像处理中的应用主要有两个方面：一是图像分割，二是目标跟踪。在图像分割中，均值漂移算法可以用来进行图像分割，其基本思想是：将图像中的每个像素点看作样本点，以该点为中心划定一个圆形区域，求取该区域内样本点的质心，即密度中心，然后以质心为新的中心点重复上述操作，直至最终收敛，最终收敛的点归为一类。在目标跟踪中，均值漂移算法可以用来进行目标跟踪，其基本思想是：将目标区域看作样本点，以该点为中心划定一个圆形区域，求取该区域内样本点的质心，即密度中心，然后以质心为新的中心点重复上述操作，直至最终收敛，最终收敛的点即为目标的位置。
### 1.3 均值漂移的实现
均值漂移算法的实现主要有两个问题：一是如何确定初始位置，二是如何确定窗口大小。对于第一个问题，一般是通过手动指定目标的位置，对于第二个问题，一般是通过目标的大小来确定窗口的大小。在OpenCV中，均值漂移算法的实现主要有两个函数：cv2.pyrMeanShiftFiltering()和cv2.meanShift()，其中，cv2.pyrMeanShiftFiltering()函数是基于图像分割的均值漂移算法，cv2.meanShift()函数是基于目标跟踪的均值漂移算法。
#### 1.3.1 基于图像分割的均值漂移算法
基于图像分割的均值漂移算法的函数原型如下：
```python
dst = cv2.pyrMeanShiftFiltering(src, sp, sr, maxLevel, termcrit)
```
其中，src表示输入图像，dst表示输出图像，sp表示空间窗口大小，sr表示色彩窗口大小，maxLevel表示金字塔的最大层数，termcrit表示迭代终止条件。该函数的返回值dst是一个与输入图像src大小相同的图像，其中，dst中的像素值表示该像素所属的类别，即最终收敛的点所属的类别。
#### 1.3.2 基于目标跟踪的均值漂移算法
基于目标跟踪的均值漂移算法的函数原型如下：
```python
retval, window = cv2.meanShift(probImage, window, termcrit)
```
其中，probImage表示目标的概率图像，window表示目标的初始位置和大小，termcrit表示迭代终止条件，retval表示是否找到目标，window表示目标的位置和大小。该函数的返回值retval表示是否找到目标，如果找到目标，retval的值为1，否则为0；window表示目标的位置和大小，如果找到目标，window的值为目标的位置和大小，否则为初始值。probImage的计算方法如下：
```python
probImage = cv2.calcBackProject(images, channels, hist, ranges, scale)
```
其中，images表示输入图像，channels表示通道索引，hist表示直方图，ranges表示像素值范围，scale表示比例因子。该函数的返回值probImage是一个与输入图像大小相同的图像，其中，probImage中的像素值表示该像素属于目标的概率。
### 1.4 均值漂移中的直方图反向投影
在mean-shift和camshift中使用反向投影,原因是:
1. 我们提供了我们要跟踪的对象的直方图。 然后，通过查找图像中的每个像素的直方图，我们可以找到图像中的每个像素的可能性。 它会为图像中的每个像素提供一个值，该值表示该像素属于我们要跟踪的对象的可能性。
2. 现在我们设置了一个阈值值，例如0到1之间的值。 然后我们过滤掉所有低于阈值的像素。 这样我们得到的图像中的所有像素都是我们要跟踪的对象的一部分。 任何其他颜色的像素都被删除了。 这就是我们在camshift中做的。



In [1]:
import numpy as np
import cv2 as cv
import argparse
filename = '../data/car.avi'

cap = cv.VideoCapture(cv.samples.findFileOrKeep(filename))
#cap = cv.VideoCapture(0)

# 视频的第一帧
ret,frame = cap.read()
# 设置窗口的初始位置
x, y, w, h = 300, 200, 100, 50 # simply hardcoded the values
track_window = (x, y, w, h)
# 设置初始ROI来追踪
roi = frame[y:y+h, x:x+w]
# 将ROI转换为HSV颜色空间
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
# 创建一个mask，其中低于lower_red和高于upper_red的区域为0，其余为1
mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
# 计算直方图 解释参数: image, channels, mask, histSize, ranges[, hist[, accumulate]]
# 第一个是要计算的图像，第二个是使用的通道，第三个是掩膜，第四个是直方图的大小，第五个是直方图中各个像素的值范围
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
# 归一化 为什么要归一化？因为直方图的值域是0-180，而mask的值域是0-255，所以需要归一化,为什么不直接指定0-255呢？
# 因为这样会使得直方图的值域变大，不利于后续的计算,但是最后还是0-255，只是中间的计算过程中会变大
# 为什么是180?因为hsv的h的值域是0-180
cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)
# 设置终止条件，可以是10次迭代，也可以至少移动1 pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
while(1):
    ret, frame = cap.read()
    if ret == True:
        # 将帧转换为HSV颜色空间
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        # 计算反向投影(直方图那一节) roi_hist是我们要查找的目标，hsv是我们搜索的图像，结果是dst
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        ret, track_window = cv.meanShift(dst, track_window, term_crit)
        # 在图像上绘制
        x,y,w,h = track_window
        img2 = cv.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv.imshow('img2',img2)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cap.release()
cv.destroyAllWindows()

## 2. Camshift
### 2.1 Camshift的原理
Camshift是一种基于均值漂移的目标跟踪算法，它的基本思想是：首先对目标进行颜色分布的建模，然后通过均值漂移算法对目标进行跟踪。Camshift算法的主要优点是：对于光照变化和目标尺寸变化具有较好的鲁棒性。Camshift算法的主要缺点是：对于目标旋转的鲁棒性较差。
### 2.2 Camshift的应用
Camshift算法主要用于目标跟踪，其基本思想是：首先对目标进行颜色分布的建模，然后通过均值漂移算法对目标进行跟踪。Camshift算法的主要优点是：对于光照变化和目标尺寸变化具有较好的鲁棒性。Camshift算法的主要缺点是：对于目标旋转的鲁棒性较差。
### 2.3 Camshift的实现
Camshift算法的实现主要有两个问题：一是如何确定初始位置，二是如何确定窗口大小。对于第一个问题，一般是通过手动指定目标的位置，对于第二个问题，一般是通过目标的大小来确定窗口的大小。在OpenCV中，Camshift算法的实现主要有两个函数：cv2.CamShift()和cv2.meanShift()，其中，cv2.CamShift()函数是基于目标跟踪的Camshift算法，cv2.meanShift()函数是基于目标跟踪的均值漂移算法。
#### 2.3.1 基于目标跟踪的Camshift算法
基于目标跟踪的Camshift算法的函数原型如下：
```python
retval, window = cv2.CamShift(probImage, window, termcrit)
```
其中，probImage表示目标的概率图像，window表示目标的初始位置和大小，termcrit表示迭代终止条件，retval表示是否找到目标，window表示目标的位置和大小。该函数的返回值retval表示是否找到目标，如果找到目标，retval的值为1，否则为0；window表示目标的位置和大小，如果找到目标，window的值为目标的位置和大小，否则为初始值。

In [2]:
import numpy as np
import cv2 as cv
import argparse
filename = '../data/car.avi'

cap = cv.VideoCapture(cv.samples.findFileOrKeep(filename))
#cap = cv.VideoCapture(0)
# 获取视频第一帧
ret,frame = cap.read()
# 设置初始窗口
x, y, w, h = 300, 200, 100, 50 # simply hardcoded the values
track_window = (x, y, w, h)
# 设置追踪的ROI窗口
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)
# 设置终止条件，可以是10次迭代，有可以至少移动1个像素
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
while(1):
    ret, frame = cap.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        # 应用camshift 到新位置
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        # 在图像上画出来
        pts = cv.boxPoints(ret)#返回值res是一个元组，其中包含了矩形的四个顶点的坐标
        pts = np.int0(pts)
        img2 = cv.polylines(frame,[pts],True, 255,2)
        cv.imshow('img2',img2)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cap.release()
cv.destroyAllWindows()

C:\Users\28678\AppData\Local\Temp\ipykernel_44112\1455791923.py:30: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  pts = np.int0(pts)
